# Создаем генератор розничного кредитного портфеля

In [1]:
import pandas as pd
import numpy as np
import random
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta

In [2]:
# !!! формирование данных разложено на этапы для лучшего понимания происходящего в ущерб скорости.
# Для увеличения скорости количество циклов свести к минимуму. Обрабатывает портфели примерно до 50тыс. кредитов - потом подтормаживает.

# определяем начальную и конечную даты периода для рэндома даты выдачи и переводим их в цифру (кол-во дней с 01.01.0001)
start_date_issue = date(2012, 1, 1).toordinal()  # дата запуска розничного кредитования
end_date = date.today().toordinal()   # текущая дата

loan = ['mortgage','car loan','car loan','car loan','consumer','consumer','consumer']   # список для рэндома типа продукта
# через значения в списке также делаем примерную процентовку по типу кредита (на скорую руку)
gen = ['male','female']   # список для расстановки пола
mar = [True, False]   # список для расстановки брака
q = 50000   # определяем количество выданных за весь период кредитов

# создаем структуру датафрейма на q строк, часть столбцов заполняем рандомом
df = pd.DataFrame({'ID': [i+1 for i in range (q)], 
                   'date_issue': [date.fromordinal(random.randint(start_date_issue, end_date)) for i in range (q)],
                   'type': [random.choice(loan) for i in range (q)],
                   'currency': [0 for i in range (q)],
                   'sum_issue': [0 for i in range (q)],
                   'term': [0 for i in range (q)],
                   'rate': [0 for i in range (q)],

                   'sum_today_cur': [0 for i in range (q)],
                   'sum_today_RUB': [0 for i in range (q)],
                   'month_payment_cur': [0 for i in range (q)],
                   'month_payment_RUB': [0 for i in range (q)],                   
                                      
                   'security': [0 for i in range (q)],
                   'secur_cost_RUB': [0 for i in range (q)],
                                                        
                   'gender': [random.choice(gen) for i in range (q)],
                   'surname': [0 for i in range (q)],
                   'name': [0 for i in range (q)],
                   'date_of_birth': [0 for i in range (q)],
                   'marriage': [random.choice(mar) for i in range (q)],
                   
                   'city': [0 for i in range (q)],
                  })

print(df.head())

   ID  date_issue      type  currency  sum_issue  term  rate  sum_today_cur  \
0   1  2014-09-16  consumer         0          0     0     0              0   
1   2  2012-09-24  consumer         0          0     0     0              0   
2   3  2015-01-15  car loan         0          0     0     0              0   
3   4  2018-10-01  car loan         0          0     0     0              0   
4   5  2017-02-22  consumer         0          0     0     0              0   

   sum_today_RUB  month_payment_cur  month_payment_RUB  security  \
0              0                  0                  0         0   
1              0                  0                  0         0   
2              0                  0                  0         0   
3              0                  0                  0         0   
4              0                  0                  0         0   

   secur_cost_RUB  gender  surname  name  date_of_birth  marriage  city  
0               0  female        0     0  

In [4]:
# Заполняем столбцы имени и фамилии
# подгружаем файл со списками фамилий и имен для заполнения
names = pd.read_csv(r'names.txt', delimiter='|')

# создаем из сериес списки имен и фамилий, параллельно убирая nan (пустые значения)
names_m = names['name_man']
names_m = list(names_m[names_m.notnull()])

names_w = names['name_woman']
names_w = list(names_w[names_w.notnull()])

surnames = names['surname']
surnames = list(surnames[surnames.notnull()])

print(names_m[:10])
print(names_w[:10])
print(surnames[:10])
print()

# заполняем столбцы фамилии и имени

for i in range(q):
    if df.loc[i, 'gender'] == 'male':
        df.loc[i, 'surname'] = random.choice(surnames)
        df.loc[i, 'name'] = random.choice(names_m)   # мужской пол заполняем фамилией и именем из мужского списка
    
    else:
        s = random.choice(surnames)   # женскую фамилию берем из общего списка и корректируем по условиям
        if s.count('ий') > 0:
            s = s.replace ('ий', 'ая', 1)
        else:
            s = (s + 'a')
        
        df.loc[i, 'surname'] = s
        df.loc[i, 'name'] = random.choice(names_w)   # имя просто берем из женского списка
        
print(df.head())

['Артём', 'Александр', 'Михаил', 'Максим', 'Иван', 'Даниил', 'Дмитрий', 'Кирилл', 'Никита', 'Егор']
['София', 'Виктория', 'Анна', 'Мария', 'Алиса', 'Анастасия', 'Дарья', 'Полина', 'Елизавета', 'Екатерина']
['Иванов', 'Смирнов', 'Кузнецов', 'Попов', 'Васильев', 'Петров', 'Соколов', 'Михайлов', 'Новиков', 'Федоров']

   ID  date_issue      type  currency  sum_issue  term  rate  sum_today_cur  \
0   1  2014-09-16  consumer         0          0     0     0              0   
1   2  2012-09-24  consumer         0          0     0     0              0   
2   3  2015-01-15  car loan         0          0     0     0              0   
3   4  2018-10-01  car loan         0          0     0     0              0   
4   5  2017-02-22  consumer         0          0     0     0              0   

   sum_today_RUB  month_payment_cur  month_payment_RUB  security  \
0              0                  0                  0         0   
1              0                  0                  0         0   
2   

In [5]:
# заполняем столбец с валютой (менее 10% портфеля - валютные)

cur = ['USD','EUR','RUB']

for i in range(q):
    if df.loc[i, 'ID']%10 == 1:
        df.loc[i, 'currency'] = random.choice(cur)
    else:
        df.loc[i, 'currency'] = 'RUB'

print(df.head(5))

   ID  date_issue      type currency  sum_issue  term  rate  sum_today_cur  \
0   1  2014-09-16  consumer      USD          0     0     0              0   
1   2  2012-09-24  consumer      RUB          0     0     0              0   
2   3  2015-01-15  car loan      RUB          0     0     0              0   
3   4  2018-10-01  car loan      RUB          0     0     0              0   
4   5  2017-02-22  consumer      RUB          0     0     0              0   

   sum_today_RUB  month_payment_cur  month_payment_RUB  security  \
0              0                  0                  0         0   
1              0                  0                  0         0   
2              0                  0                  0         0   
3              0                  0                  0         0   
4              0                  0                  0         0   

   secur_cost_RUB  gender   surname   name  date_of_birth  marriage  city  
0               0  female  Гладковa  Лилия    

In [6]:
# заполняем дату рождения
 
# определяем начальную дату периода для рэндома даты рождения и переводим в цифру (кол-во дней с 01.01.0001)
start_date_birth = date(1953, 1, 1).toordinal()  # возможная дата рождения самого пожилого заемщика

for i in range(q):
    d = df.loc[i, 'date_issue'].toordinal() - (20*365)   # определяем самую позднюю дату рождения заемщика, так, чтобы на момент выдачи кредита ему было не менее 20 лет
    df.loc[i, 'date_of_birth'] = date.fromordinal(random.randint(start_date_birth, d))

    
print(df.head(5))

   ID  date_issue      type currency  sum_issue  term  rate  sum_today_cur  \
0   1  2014-09-16  consumer      USD          0     0     0              0   
1   2  2012-09-24  consumer      RUB          0     0     0              0   
2   3  2015-01-15  car loan      RUB          0     0     0              0   
3   4  2018-10-01  car loan      RUB          0     0     0              0   
4   5  2017-02-22  consumer      RUB          0     0     0              0   

   sum_today_RUB  month_payment_cur  month_payment_RUB  security  \
0              0                  0                  0         0   
1              0                  0                  0         0   
2              0                  0                  0         0   
3              0                  0                  0         0   
4              0                  0                  0         0   

   secur_cost_RUB  gender   surname   name date_of_birth  marriage  city  
0               0  female  Гладковa  Лилия    1

In [7]:
# заполняем сумму выданного кредита в валюте кредита в соответствии с лимитами кредитной политики:
# Ипотека RUB - ОТ 500т. До 30 млн.
# Ипотека USD - ОТ 10т. До 600т.
# Ипотека EUR - ОТ 8т. До 500т.
# Потреб RUB - ОТ 100т. До 1млн.
# Потреб USD - ОТ 2т. До 20т.
# Потреб EUR - ОТ 1,5т. До 15т.
# Авто RUB - ОТ 200т. До 3 млн.
# Авто USD - ОТ 2т. До 60т.
# Авто EUR - ОТ 1,5т. До 50т.

# заполняем % ставку в соответствии с политикой:
# Ипотека RUR - 9 -12
# Ипотека USD - 3-5
# Ипотека EUR - 3 -5
# Потреб RUR - 18-25
# Потреб USD - 9-11
# Потреб EUR - 8-11
# Авто RUR - 14-18
# Авто USD - 7-10
# Авто EUR - 6-10


for i in range(q):
    # рублевые:
    if (df.loc[i, 'type'] == 'mortgage' and df.loc[i, 'currency'] == 'RUB'):
        df.loc[i, 'sum_issue'] = random.randint(50,3000)*10000
        df.loc[i, 'rate'] = random.randint(9,12)
        
    elif (df.loc[i, 'type'] == 'consumer' and df.loc[i, 'currency'] == 'RUB'):
        df.loc[i, 'sum_issue'] = random.randint(10,100)*10000
        df.loc[i, 'rate'] = random.randint(18,25)
        
    elif (df.loc[i, 'type'] == 'car loan' and df.loc[i, 'currency'] == 'RUB'):
        df.loc[i, 'sum_issue'] = random.randint(20,300)*10000
        df.loc[i, 'rate'] = random.randint(14,18)
        
    # долларовые:
    elif (df.loc[i, 'type'] == 'mortgage' and df.loc[i, 'currency'] == 'USD'):
        df.loc[i, 'sum_issue'] = random.randint(100,6000)*100
        df.loc[i, 'rate'] = random.randint(3,5)
        
    elif (df.loc[i, 'type'] == 'consumer' and df.loc[i, 'currency'] == 'USD'):
        df.loc[i, 'sum_issue'] = random.randint(20,200)*100
        df.loc[i, 'rate'] = random.randint(9,11)
        
    elif (df.loc[i, 'type'] == 'car loan' and df.loc[i, 'currency'] == 'USD'):
        df.loc[i, 'sum_issue'] = random.randint(20,600)*100
        df.loc[i, 'rate'] = random.randint(7,10)
        
    # в евро:
    elif (df.loc[i, 'type'] == 'mortgage' and df.loc[i, 'currency'] == 'EUR'):
        df.loc[i, 'sum_issue'] = random.randint(80,5000)*100
        df.loc[i, 'rate'] = random.randint(3,5)
        
    elif (df.loc[i, 'type'] == 'consumer' and df.loc[i, 'currency'] == 'EUR'):
        df.loc[i, 'sum_issue'] = random.randint(15,150)*100
        df.loc[i, 'rate'] = random.randint(8,11)
        
    elif (df.loc[i, 'type'] == 'car loan' and df.loc[i, 'currency'] == 'EUR'):
        df.loc[i, 'sum_issue'] = random.randint(15,500)*100
        df.loc[i, 'rate'] = random.randint(6,10)


In [8]:
# заполняем срок выдачи в соответствии с кредитной политикой:
# Ипотека 120-300 мес.
# Автокредит 12-84 мес.
# Потребительский беззалоговый 12-60 мес.

for i in range(q):
    if df.loc[i, 'type'] == 'mortgage':
        df.loc[i, 'term'] = random.randint(10,25)*12
    elif df.loc[i, 'type'] == 'consumer':
        df.loc[i, 'term'] = random.randint(2,10)*6
    elif df.loc[i, 'type'] == 'car loan':
        df.loc[i, 'term'] = random.randint(2,14)*6


In [9]:
# заполняем город выдачи (20% портфеля - Новосиб, Екб, 40% - Спб/ Мск, остальное Мск)

cities1 = ['Москва','Санкт-Петербург','Новосибирск','Екатеринбург']
cities2 = ['Новосибирск','Екатеринбург']
cities3 = ['Москва','Санкт-Петербург']


for i in range(q):
    if df.loc[i, 'ID']%10 == 1:
        df.loc[i, 'city'] = random.choice(cities1)   # под этим ID вся ипотека, ее раскидываем равномерно
    
    elif (df.loc[i, 'ID']%10 == 2 or df.loc[i, 'ID']%10 == 3):   # 20% Новосиб, Екб
        df.loc[i, 'city'] = random.choice(cities2)
    
    elif (df.loc[i, 'ID']%10 == 4 or df.loc[i, 'ID']%10 == 5 or df.loc[i, 'ID']%10 == 6 or df.loc[i, 'ID']%10 == 7):   # 40% - Спб/ Мск
        df.loc[i, 'city'] = random.choice(cities3)

    else:
        df.loc[i, 'city'] = 'Москва'


In [10]:
# Рассчитываем:
# 1. сумму ежемесячного платежа (путем простого деления суммы выдачи на количество периодов - платежи не аннуитетные (дифференцированные)) в валюте
# 2. сумму остатка долга на сегодня (вычет уже выплаченных по графику платежей из суммы выдачи) в валюте
# 3 и 4 - значения 1 и 2 в рублях для удобства расчетов

#from datetime import date
#from dateutil.relativedelta import relativedelta

usd = 77
eur = 91

for i in range(q):
    df.loc[i, 'month_payment_cur'] = round(df.loc[i, 'sum_issue'] / df.loc[i, 'term'])   # высчитываем ежемесячный платеж (ОД) в валюте кредита и округляем до целого
    
    delta = relativedelta(date.today(), df.loc[i, 'date_issue'])   # считаем дельту между сегодяшней датой и датой выдачи ссуды
    d1 = delta.years * 12 + delta.months   # преобразуем дельту в точное количество полных прошедших месяцев
    df.loc[i, 'sum_today_cur'] = round(df.loc[i, 'sum_issue'] - (df.loc[i, 'month_payment_cur'] * d1))   # из суммы выдачи вычитаем сумму уже выплаченных платежей и округляем до целого
    
    if df.loc[i, 'sum_today_cur'] <= 0:   # если кредит уже погашен (выплачено основного долга больше, чем получено)
        df.loc[i, 'sum_today_cur'] = 0
    
    
    # дальше преобразуем полученные значения в рубли по указанному курсу и округляем до целого
    if df.loc[i, 'currency'] == 'USD':
        df.loc[i, 'month_payment_RUB'] = round(df.loc[i, 'month_payment_cur'] * usd)
        df.loc[i, 'sum_today_RUB'] = round(df.loc[i, 'sum_today_cur'] * usd)

    elif df.loc[i, 'currency'] == 'EUR':
        df.loc[i, 'month_payment_RUB'] = round(df.loc[i, 'month_payment_cur'] * eur)  
        df.loc[i, 'sum_today_RUB'] = round(df.loc[i, 'sum_today_cur'] * eur)
    
    else:
        df.loc[i, 'month_payment_RUB'] = df.loc[i, 'month_payment_cur']  
        df.loc[i, 'sum_today_RUB'] = df.loc[i, 'sum_today_cur']

        
print(df.head(5))

   ID  date_issue      type currency  sum_issue  term  rate  sum_today_cur  \
0   1  2014-09-16  consumer      USD      19300    48     9            0.0   
1   2  2012-09-24  consumer      RUB     930000    60    21            0.0   
2   3  2015-01-15  car loan      RUB     520000    84    17        86700.0   
3   4  2018-10-01  car loan      RUB    1330000    18    18            0.0   
4   5  2017-02-22  consumer      RUB     580000    42    20            0.0   

   sum_today_RUB  month_payment_cur  month_payment_RUB  security  \
0            0.0              402.0            30954.0         0   
1            0.0            15500.0            15500.0         0   
2        86700.0             6190.0             6190.0         0   
3            0.0            73889.0            73889.0         0   
4            0.0            13810.0            13810.0         0   

   secur_cost_RUB  gender   surname   name date_of_birth  marriage  \
0               0  female  Гладковa  Лилия    1973-1

In [13]:
# выгружаем даннные в csv (в текущую папку)
df.to_csv(str(date.today()) + r' Loan portfolio.txt', sep="|", encoding = 'utf-8')              

# Ниже примеры из генератора

In [216]:
# примеры работы с датами. Пример 1

from datetime import date
from dateutil.relativedelta import relativedelta

one_month = date.today() - relativedelta(months=5)
print(one_month)
print()


# пример 2 - работа с парсером, который считывает дату из строки

from dateutil.parser import parse

date1 = '21.10.2020'
date2 = 20181105

d1 = parse(str(date1))
d2 = parse(str(date2))
delta = relativedelta(d1, d2)

print(delta.years)
print(delta.months)
print(delta.days)

2020-05-21

1
11
16


In [ ]:
# пример работы алгоритма с вычетом даты для рандома

start_date = date(2012, 1, 1).toordinal()
end_date = date.today().toordinal()
random_day = date.fromordinal(random.randint(start_date, end_date))

print(start_date,end_date,random_day)

In [ ]:
#data = pd.DataFrame([[0 for i in range (15)],
#                  [0 for i in range (15)],
#                  [0 for i in range (15)]], 
#columns=['ID','date_issue','sum_issue','currency','term','type','sum_today_cur','sum_today_RUB','rate','security','secur_cost_RUR','lender','date_birth','marriage','city'])
#
#print (data)